In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [25]:
from IPython.display import clear_output

In [2]:
breweries_file = r'../data/bigData/breweries.csv'

In [3]:
breweries = pd.read_csv (breweries_file)

In [4]:
breweries.columns

Index(['id', 'name', 'city', 'state', 'country', 'notes', 'types'], dtype='object')

In [5]:
cities = breweries[["country","city"]].drop_duplicates()
cities = cities.values
cities[100]

array(['US', 'Effingham'], dtype=object)

In [6]:
len(cities)

12077

## geonames database

In [35]:
names = ["id","name","name2","alias","lat","long","d1","d2","iso"]
namesuseless =  [str(i) for i in range(10)]
cities_loc = pd.read_csv("../../data/cities500.txt",sep="\t",names=names+namesuseless)
cities_loc = cities_loc[["id","name","name2","alias","lat","long","iso","6"]]
cities_loc.columns

/home/loic/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['id', 'name', 'name2', 'alias', 'lat', 'long', 'iso', '6'], dtype='object')

In [39]:
print(cities_loc.size)
cities_loc.head()

1558312


,id,name,name2,alias,lat,long,iso,6
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,AD,NaN
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,AD,NaN
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,AD,NaN
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,AD,2050.0
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,AD,NaN


In [132]:
from geopy.geocoders import Nominatim
geolocator =Nominatim()



/home/loic/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [157]:
i=0
err = 0
entries=0

try:
    cities_pos
except NameError:
    cities_pos = breweries[["country","city"]].drop_duplicates()
    cities_pos = cities_pos.values.tolist()
    

alias_nonan = cities_loc.dropna(subset=["alias"])





for s_city in cities_pos:
    i+=1
    if i%100 == 0:
        print("{:3.2f}%".format(100*i/len(cities)),end="\r")
    
    if len(s_city) ==4:
        continue
    
    state = s_city[0]
    city = s_city[1]
    
    if type(city)!=str:
        continue
    
    
    filter_name = cities_loc[cities_loc["name"] == city]
    if type(state)==str:
        filter_country = filter_name[filter_name["iso"] == state]
    else:
        filter_country = filter_name
    
    done = False
    
    while(not done):
        if len(filter_country)==0:
            filter_name = alias_nonan[alias_nonan["alias"].str.contains(city)]
            filter_country = filter_name[filter_name["iso"] ==state]

        if len(filter_country)==1:
            entries+=1
            s_city += [ str(filter_country['lat'].values[0])]
            s_city += [ str(filter_country['long'].values[0])]
            
            done= True

        if len(filter_country)>1:
            
            
            filter_country = filter_country.sort_values("6",ascending=False)

            s_city += [ str(filter_country['lat'].values[0])]
            s_city += [ str(filter_country['long'].values[0])]
            done=True

        if len(filter_country)==0:
            location = None
            if type(state)==str:
                location = geolocator.geocode(city,country_codes=[state])
            if location is not None:
                entries+=1
                s_city += [ str(location.latitude)]
                s_city += [ str(location.longitude)]
                done = True
            else:
                err+=1
                print("{:3.2f}%".format(100*i/len(cities)))
            
                print("no data for:",city, "  ", state)

                print("""Search for  (or enter "" to enter lat and longitude manually)""")
                city = input()

                if(city ==""):
                    print("latitude")
                    lat = input()
                    print("longitude")
                    long = input()
                    done =True
                    entries+=1
                    s_city += [ lat]
                    s_city += [ long]
                    
                clear_output()
                


   
        
        
        
    
print()
print(entries)
print(err)
    

99.36%
0
0


In [156]:
import csv
from datetime import datetime

today = datetime.now()
f = open('lat_long_'+today.strftime("%d-%m-%Y_%H:%M")+'.csv', 'w')
cols = ["Country","City","lat","long"]
with f:
    writer = csv.writer(f)
    writer.writerow(cols)
    writer.writerows(cities_pos)
    
    

In [146]:


str(today.strftime("%d-%m-%Y_%H"))

'03-05-2020_18'

In [99]:
cities_pos[2050] = ['IT', 'Lemignano', '44.7711928', '10.2633709']
for i,s_city in enumerate(cities_pos):
    if len(s_city)>4:
        print(i," ",s_city)